In [46]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Dense,Dropout,RandomRotation,RandomFlip,MaxPooling2D,Conv2D,Flatten,Rescaling,RandomZoom
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from PIL import Image
import os

In [7]:
img_skip =0
for folder_name in('Blight','Common_Rust','Gray_Leaf_spot','Healthy'):
    folder_path=os.path.join('data',folder_name)
    for f_name in os.listdir(folder_path):
        f_path =os.path.join(folder_path,f_name)
        try:
            img_bytes=tf.io.read_file(f_path)
            tf.io.decode_image(img_bytes)
           
        except Exception as e:
            print(f"Bad file found and deleted on {f_path}")
            img_skip +=1
            os.remove(f_path)

print(f"picture deleted due to corrupt {img_skip}")

picture deleted due to corrupt 0


In [4]:
directory ='data'
training_ds =tf.keras.utils.image_dataset_from_directory(
    directory,
    subset='training',
    seed=3457,
    validation_split=0.2,
    image_size =(180,180),
    batch_size=20
)
validation_ds=tf.keras.utils.image_dataset_from_directory(
    directory,
    seed=3457,
    subset='validation',
    validation_split=0.2,
    image_size=(180,180),
    batch_size=20,
    

)

Found 4188 files belonging to 4 classes.
Using 3351 files for training.
Found 4188 files belonging to 4 classes.
Using 837 files for validation.


In [50]:
model=Sequential([
    Input(shape=(180,180,3)),
    Rescaling(1./255),
    RandomFlip('horizontal'),
    RandomRotation(0.3),
    RandomZoom(height_factor=0.3),
    Conv2D(32,3,activation='relu'),
    Dropout(0.3),
    MaxPooling2D(),
    Conv2D(64,3,activation='relu'),
    MaxPooling2D(),
    Conv2D(122,3,activation='relu'),
    Dropout(0.3),
    MaxPooling2D(),
    Conv2D(124,3,activation='relu'),
    Flatten(),
    Dense(124,activation='relu'),
    Dropout(0.3),
    Dense(4,activation='softmax')
])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_4 (Rescaling)         │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_flip_4 (RandomFlip)      │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation_4               │ (None, 180, 180, 3)    │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_zoom_1 (RandomZoom)      │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 178, 178, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 178, 178, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 89, 89, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 87, 87, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 43, 43, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 41, 41, 122)    │        70,394 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 41, 41, 122)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 20, 20, 122)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 18, 18, 124)    │       136,276 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 40176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 124)            │     4,981,948 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 124)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           500 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,208,510 (19.87 MB)

 Trainable params: 5,208,510 (19.87 MB)

 Non-trainable params: 0 (0.00 B)

In [51]:
model.fit(
    training_ds,
    validation_data=validation_ds,
    epochs=10,
    callbacks=[EarlyStopping(monitor='val_accuracy',patience=2,mode='max'),ModelCheckpoint('best_maize_disease_ditect_model.keras',save_best_only=True,monitor='val_loss',mode='min')],
    verbose=1
)

Epoch 1/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 231s 1s/step - accuracy: 0.5639 - loss: 1.1255 - val_accuracy: 0.7539 - val_loss: 0.6427
Epoch 2/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 119s 697ms/step - accuracy: 0.7964 - loss: 0.5494 - val_accuracy: 0.7802 - val_loss: 0.5016
Epoch 3/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 105s 618ms/step - accuracy: 0.8191 - loss: 0.4675 - val_accuracy: 0.8184 - val_loss: 0.4252
Epoch 4/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 107s 633ms/step - accuracy: 0.8274 - loss: 0.4110 - val_accuracy: 0.8100 - val_loss: 0.3939
Epoch 5/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 102s 603ms/step - accuracy: 0.8353 - loss: 0.3868 - val_accuracy: 0.8387 - val_loss: 0.4048
Epoch 6/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 102s 604ms/step - accuracy: 0.8379 - loss: 0.3856 - val_accuracy: 0.8722 - val_loss: 0.3673
Epoch 7/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 101s 598ms/step - accuracy: 0.8463 - loss: 0.3961 - val_accuracy: 0.8232 - val_loss: 0.4454
Epoch 8/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 102s 607ms/step - accuracy: 0.8536 - lo

In [53]:
saved_model=tf.keras.models.load_model("best_maize_disease_ditect_model.keras")

In [54]:
img =tf.keras.utils.load_img('heal_maize2.jpg',target_size=(180,180,3))
img_array =tf.keras.utils.img_to_array(img)
img_batch =np.expand_dims(img_array,axis=0)
pred=saved_model.predict(img_batch)

1/1 ━━━━━━━━━━━━━━━━━━━━ 13s 13s/step


In [55]:
classes=['Blight','Common_Rust','Gray_leaf_sport','Health']
index = np.argmax(pred)
classes[int(index)]


'Gray_leaf_sport'